### **RNN for sentiment Analyisis**

Uses imdb dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
import numpy as np


import sys
sys.stderr = open('err.txt', 'w')

2025-12-15 11:22:33.796705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 11:22:33.856508: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/jaumemanero/RL/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_c

In [2]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

Using device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:52:00.0, compute capability: 8.6


#### **Load IMDB Dataset from Keras Datasets**

In [3]:
vocab_size = 10000  # Keep the top 10,000 words
max_length = 100  # Maximum review length
embedding_dim = 32  # Word embedding vector size

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)


#### **Prepare Dataset padding, etc..**

In [4]:
train_data = pad_sequences(train_data, maxlen=max_length, padding='post', truncating='post')
test_data = pad_sequences(test_data, maxlen=max_length, padding='post', truncating='post')

#### **Build RNN model**

In [5]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.LSTM(64, return_sequences=True),  # LSTM layer with 64 units
    keras.layers.LSTM(32),  # Another LSTM layer with 32 units
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Output: Probability of being positive
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
epochs = 15
batch_size = 64

model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data, test_labels))


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 44ms/step - accuracy: 0.7436 - loss: 0.5080 - val_accuracy: 0.7904 - val_loss: 0.4781
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - accuracy: 0.8454 - loss: 0.3850 - val_accuracy: 0.7986 - val_loss: 0.4602
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.8918 - loss: 0.2838 - val_accuracy: 0.8036 - val_loss: 0.4536
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 48ms/step - accuracy: 0.9136 - loss: 0.2380 - val_accuracy: 0.8040 - val_loss: 0.4800
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 50ms/step - accuracy: 0.9284 - loss: 0.2021 - val_accuracy: 0.7826 - val_loss: 0.5417
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.9386 - loss: 0.1740 - val_accuracy: 0.7845 - val_loss: 0.6592
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 50ms/step - accuracy: 0.8430 - loss: 0.3060 - val_accuracy: 0.6706 - val_loss: 0.6297
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - accuracy: 0.5241 - loss: 0.6855 - 

In [7]:
scores = model.evaluate(test_data, test_labels, verbose=0)

print('Test accuracy:', scores[1])

Test accuracy: 0.7757599949836731


In [8]:

word_index = imdb.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}

# Function to encode a custom sentence
def encode_sentence(sentence):
    words = sentence.lower().split()  # Tokenize sentence
    encoded = [word_index.get(word, 2) for word in words]  # 2 = <UNK> for unknown words
    return pad_sequences([encoded], maxlen=max_length, padding='post')  # Pad for model




1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
# --- 5. Test Model with a Custom Sentence ---
test_sentence = "This movie was fantastic and thrilling!"
encoded_sentence = encode_sentence(test_sentence)

# Predict sentiment
prediction = model.predict(encoded_sentence)[0][0]
sentiment = "Positive" if prediction > 0.5 else "Negative"

print(f"Sentence: {test_sentence}")
print(f"Sentiment: {sentiment} (Confidence: {prediction:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Sentence: This movie was fantastic and thrilling!
Sentiment: Positive (Confidence: 0.8967)


In [10]:
# --- 5. Test Model with a Custom Sentence ---
test_sentence = "This movie was really bad"
encoded_sentence = encode_sentence(test_sentence)

# Predict sentiment
prediction = model.predict(encoded_sentence)[0][0]
sentiment = "Positive" if prediction > 0.5 else "Negative"

print(f"Sentence: {test_sentence}")
print(f"Sentiment: {sentiment} (Confidence: {prediction:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Sentence: This movie was really bad
Sentiment: Negative (Confidence: 0.3869)


In [11]:
# --- 5. Test Model with a Custom Sentence ---
test_sentence = "I went to the movie theater with high expectations but I was bored to death"
encoded_sentence = encode_sentence(test_sentence)

# Predict sentiment
prediction = model.predict(encoded_sentence)[0][0]
sentiment = "Positive" if prediction > 0.5 else "Negative"

print(f"Sentence: {test_sentence}")
print(f"Sentiment: {sentiment} (Confidence: {prediction:.4f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Sentence: I went to the movie theater with high expectations but I was bored to death
Sentiment: Negative (Confidence: 0.0895)


In [12]:
import session_info
session_info.show(html=False)

-----
keras               3.12.0
numpy               2.2.6
session_info        v1.0.1
tensorflow          2.20.0
-----
IPython             8.37.0
jupyter_client      8.6.3
jupyter_core        5.8.1
jupyterlab          4.4.5
notebook            7.4.4
-----
Python 3.10.12 (main, Nov  4 2025, 08:48:33) [GCC 11.4.0]
Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
-----
Session information updated at 2025-12-15 11:27
